In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize
from collections import defaultdict
from torch.utils.data import random_split

In [ ]:
print(torch.cuda.is_available())
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transforms_train,
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transforms_,
)

train_size = int(0.5*len(training_data))
val_size = len(training_data) - train_size
training_data, validation_data = random_split(training_data, [train_size, val_size])

In [ ]:
batch_size = 12


# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size,shuffle = True)
test_dataloader = DataLoader(test_data, batch_size=100)

for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Define model
class Model_NN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super().__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=int(32), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(32)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(32), out_channels=int(64), kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=int(64), out_channels=int(128), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(128)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(128), out_channels=int(128), kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),

            # Conv Layer block 3
            nn.Conv2d(in_channels=int(128), out_channels=int(256), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(256)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(256), out_channels=int(256), kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_x_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(4096, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(int(512), 10),
            nn.LogSoftmax(dim=1)
        )
        
        self.fc_g_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(4096, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),
            nn.Linear(int(512), 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        #x = self.fc_layer(x)


        y = self.fc_x_layer(x)

        self.g = self.fc_g_layer(x)

        #x = torch.stack((self.x1,self.g))
        self.g = (self.g).float()
        y = y.float()

        

        return y
    def get_g(self):
        return self.g
      

model = Model_NN()
model = model.to(dev)
print(model)

In [ ]:
def freeze_params(model, name = None):
    for n,param in model.named_parameters():
        if name ==None:
            param.requires_grad = False
        elif name in n:
            param.requires_grad = False
def unfreeze_params(model, name = None):
    for n,param in model.named_parameters():
        if name == None:
            param.requires_grad = True
        elif name in n:
            param.requires_grad = True

In [ ]:

class rejection_loss(nn.Module):
    def __init__(self,criterion, def_loss, no_grad= True):
        super().__init__()
        self.L0 = def_loss
        self.criterion = criterion
        self.no_grad = no_grad
 
    def forward(self, y_pred,g,y_true):
        
        loss = g*self.criterion(y_pred,y_true)+(1-g)*self.L0
        loss = torch.mean(loss)
        
        return loss
    def update_L0(self,new_L0):
        if self.no_grad:
            with torch.no_grad():
                self.L0 = new_L0
        else: self.L0 = new_L0

In [ ]:

def predicted_class(y_pred):
    '''Returns the predicted class for a given softmax output.'''
    if y_pred.shape[-1] == 1:
        y_pred = y_pred.view(-1)
        y_pred = (y_pred>0.5).float()
        
    else:
        y_pred = torch.max(y_pred, 1)[1]
    return y_pred
def correct_class(y_pred,y_true):
    '''Returns a bool tensor indicating if each prediction is correct'''

    y_pred = predicted_class(y_pred)
    correct = (y_pred==y_true)
    
    return correct
def correct_total(y_pred,y_true):
    '''Returns the number of correct predictions in a batch where dk_mask=0'''
    correct = correct_class(y_pred,y_true)
    correct_total = torch.sum(correct).item()
    return correct_total


In [ ]:
def model_acc_g(model,data):
    '''Returns the total accuracy of model in some dataset'''
    loss_criterion = nn.NLLLoss(reduction = 'sum')
    model.eval()
    dev = next(model.parameters()).device
    total = 0
    correct= 0
    g_i = 0
    bce = 0
    with torch.no_grad():
        for image,label in data:
            image,label = image.to(dev), label.to(dev)
            output = model(image)
            g = model.get_g()
            total += label.size(0)
            correct += correct_total(output,label)
            g_i += torch.sum(g).item()
            bce += loss_criterion(output,label).item()
        
    return (correct/total),g_i/total, bce/total

In [ ]:
def model_eval_layers(layer):
    layer.eval()
def model_train_layers(layer):
    layer.train()
    

In [ ]:
model = Model_NN()
model = model.to(dev)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

L0_init = 0
#loss_criterion = rejection_loss(CrossEntropy,L0_init)

d_mean = defaultdict(list)
d_max = defaultdict(list)
bce_mean = []
L0_list = []
g_mean = []
mcp_list = []
acc_list = []

model.train()

loss_criterion = nn.NLLLoss(reduction = 'mean')
freeze_params(model,'fc_g')
n_epochs = 50
for epoch in range(n_epochs):
    total = 0
    correct = 0
    for image,label in train_dataloader:
        image,label = image.to(dev), label.to(dev)
        optimizer.zero_grad()
        output = model(image)
        #g = model.get_g()
        
        loss = loss_criterion(output,label)
        loss.backward()
        optimizer.step()
        
        #L0_list.append(0)
        mcp_list.append((torch.mean(torch.max(torch.exp(output),dim=1)[0]).item()))
        bce_mean.append(loss_criterion(output,label).item())
        
        #g_mean.append(torch.mean(g).item())
        #g_mm.append(np.mean(g_mean))

        total += label.size(0)
        correct += correct_total(output,label)
        acc_list.append(correct/total)
acc, g_i, bce = model_acc_g(model,validation_dataloader) #training_data   
loss_criterion = nn.NLLLoss(reduction = 'none')
freeze_params(model)
unfreeze_params(model,'fc_g')
model_eval_layers(model.conv_layer)
model_eval_layers(model.fc_x_layer)
loss_fn = rejection_loss(loss_criterion,bce,no_grad= True)#np.mean(bce_mean[-4166:]),no_grad= False)
acc_m = []
g_m = []
bce_m = []
for epoch in range(5*n_epochs):
    correct = 0
    total = 0
    for image,label in validation_dataloader:
        image,label = image.to(dev), label.to(dev)
        optimizer.zero_grad()
        output = model(image)
        
        #bce_mean.append(torch.mean(loss_criterion(output,label)).item())
        #loss_fn.update_L0(np.mean(bce_mean[-4166:]))
        
        g = model.get_g()
        loss = loss_fn(output,g,label)
        loss.backward()
        optimizer.step()
        
        L0_list.append(loss_fn.L0)
        bce_mean.append(torch.mean(loss_criterion(output,label)).item())
        g_mean.append(torch.mean(g).item())
        mcp_list.append((torch.mean(torch.max(torch.exp(output),dim=1)[0]).item()))
        
        #if (i%5000) == 0:
        #loss_fn.update_L0(np.mean(bce_mean[-4166:]))

        #loss_criterion.update_L0(torch.mean(CrossEntropy(output,label)).item())

        total += label.size(0)
        correct += correct_total(output,label)
        acc_list.append(correct/total)
    acc, g_i, bce = model_acc_g(model,validation_dataloader)
    acc_m.append(acc)
    g_m.append(g_i)
    bce_m.append(bce)
        
        
'''        for n,p in model.named_parameters():
            if(p.requires_grad) and ("bias" not in n):

                d_mean[n].append(p.grad.abs().mean().item())
                d_max[n].append(p.grad.abs().max().item())'''
                
                


In [ ]:
plt.plot(np.arange(len(g_m)),g_m,label = 'Média de g',lw=3)
plt.plot(np.arange(len(acc_m)),acc_m,label = 'Acurácia',lw=2)
plt.legend()
plt.show()
plt.plot(np.arange(len(bce_m)),bce_m,label = 'BCE',lw=1)
plt.plot(L0_list[0:250], label = 'L0',lw = 4)
plt.legend()
plt.show()

In [ ]:
def round_decimal(y,n_digits = 1):
    '''Round a tensor with n_digits digits'''
    rounded = np.around(y,1)
    return rounded

In [ ]:
listg = []
listbce = []
g_dict = defaultdict(list)
for i in round_decimal(np.arange(0,1.1,0.1)):
    g_dict[i] = [0,0]
for image,label in validation_dataloader:
    image,label = image.to(dev), label.to(dev)
    output = model(image)
    bce = loss_criterion(output,label).cpu().detach().numpy()
    g = model.get_g().view(-1).tolist()
    g = round_decimal(g)
    for i in g_dict:
        g_dict[i][0] += np.sum(bce[g==i])
        g_dict[i][1] += np.sum(g==i)
    listbce.extend(bce)
    listg.extend(g)

In [ ]:
for i in g_dict:
    acc=g_dict[i][0]/g_dict[i][1]
    plt.bar(i,acc,width=0.1)
plt.show()

In [ ]:
for image,label in test_dataloader:
    image,label = image.to(dev), label.to(dev)
    output = model(image)
    bce = loss_criterion(output,label).tolist()
    g = model.get_g().tolist()
    
    break

In [ ]:
#Colocar MCP e TCP também
plt.plot(np.arange(len(bce_mean)),bce_mean,label = 'Média de BCE',lw=0.5)
plt.plot(np.arange(len(g_mean)),g_mean,label = 'Média de g',lw=3)
#plt.plot(it_list,acc_list,label = 'Acurácia',lw=4)
plt.plot(np.arange(len(L0_list)),L0_list,label = 'L0',lw=2)
#plt.plot(it_list,(np.array(bce_mean)-np.array(L0_list)),label = 'BCE - L0',lw=2)
plt.legend()
plt.grid()
plt.title('Médias do Output')
plt.show()


'''for n in d_mean:
    plt.bar(np.array(it_list),d_max[n],color="b", label = 'Max (grad)')
    plt.plot(np.array(it_list),d_mean[n],color="g", label = 'Média do grad')
    plt.legend()
    plt.grid()
    plt.title('Camada: %s'%n)
    plt.show()'''

print('Acurácia de treino = ',model_acc(model,train_dataloader), 'Acurácia de teste = ',model_acc(model,test_dataloader))


plt.plot(np.arange(len(g_mean)),g_mean,label = 'Média de g',lw=3)
plt.plot(np.arange(len(acc_list)),acc_list,label = 'Acurácia',lw=4)
plt.plot(np.arange(len(L0_list)),L0_list,label = 'L0',lw=2)
#plt.plot(it_list,(np.array(bce_mean)-np.array(L0_list)),label = 'BCE - L0',lw=2)
plt.legend()
plt.grid()
plt.title('Médias do Output')
plt.show()


In [ ]:
plt.plot(np.arange(len(g_mean)),g_mean,label = 'Média de g',lw=3)
plt.plot(np.arange(len(L0_list)),L0_list,label = 'L0',lw=2)
#plt.plot(it_list,(np.array(bce_mean)-np.array(L0_list)),label = 'BCE - L0',lw=2)
plt.legend()
plt.grid()
plt.title('Médias do Output')
plt.show()


In [ ]:
PATH = r'/home/luis-felipe/data/output'
file = PATH + '/model_bad_big.csv'
a = np.asarray([bce_mean,mcp_list,acc_list])
np.savetxt(file, a, delimiter=",")

file = PATH + '/model_bad_small.csv'
a = np.asarray([ g_mean, L0_list])
np.savetxt(file, a, delimiter=",")

# Specify a path
PATH = r'/home/luis-felipe/torch_models/test_learned_unc_bad.pt'

# Save
torch.save(model.state_dict(), PATH)

In [ ]:
dif_m = []
n = 500
for j in range(0,len(L0_list),n):
    diff = np.mean(((bce_mean)[len(bce_mean)-i:])[j:j+n-1])-np.mean(L0_list[j:j+n-1])
    dif_m.append(diff)
    
plt.plot(range(len(dif_m)),dif_m) 

In [ ]:
model.eval()
g_m_test = []
g_min = []
g_max = []
for image,label in test_dataloader:
    image,label = image.to(dev), label.to(dev)
    y = (model(image))
    g = model.get_g()
    g_m_test.append(torch.mean(g).item())
    g_min.append(torch.min(g).item())
    g_max.append(torch.max(g).item())
plt.plot(g_m_test)
plt.plot(g_min,'ro')
plt.plot(g_max,'gx')
    

In [ ]:
model.eval()
mcp_m_test = []
mcp_min = []
mcp_max = []
for image,label in test_dataloader:
    image,label = image.to(dev), label.to(dev)
    y = torch.exp(model(image))
    MCP = torch.max(y,dim=1)[0]
    mcp_m_test.append(torch.mean(MCP).item())
    mcp_min.append(torch.min(MCP).item())
    mcp_max.append(torch.max(MCP).item())
plt.plot(mcp_m_test)
plt.plot(mcp_min,'ro')
plt.plot(mcp_max,'gx')
plt.ylim(0.8,1)

In [18]:
print('oi')

oi
